In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

df = pd.read_csv("/content/clean_data.csv")
df = df.iloc[:,[2,4]]
df.head()

,label,clean_tweet
0,0,father dysfunct selfish drag kid dysfunction. ...
1,0,thank #lyft credit can't use caus offer wheelc...
2,0,bihday majesti
3,0,#model love u take u time urd+-!!! dddd d|d|d|
4,0,factsguide: societi #motiv


In [ ]:
df.dropna(subset = ["clean_tweet"], inplace=True)

In [ ]:
def remover(my_string = ""):
  values = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ012345678 ")
  for item in my_string:
    if item not in values:
      my_string = my_string.replace(item, "")
  return my_string
df.loc[:,'clean'] = [remover(x) for x in df.clean_tweet]

In [ ]:
df = df.iloc[:,[0,2]]
dff = df.drop(['label'], axis = 1)
X_temp, X_test, y_temp, y_test = train_test_split(dff, list(df.label), test_size=0.1)

In [ ]:
X_temp['label'] = y_temp
nonhate = X_temp[X_temp['label'] == 0]
hate = X_temp[X_temp.label == 1]
nonhatesample = nonhate.sample(n = hate.shape[0])
ds = pd.concat([hate, nonhatesample], axis = 0)
ds.to_csv("/content/dataset_train.csv")
ds_temp = ds
testdf = X_test
testdf['label'] = y_test
ds = pd.concat([ds_temp, testdf], axis = 0)

In [ ]:
ds.head()

,clean,label
30526,still root society attitud toward black women ...,1
21256,pull chain stoptalk sob auspol,1
15314,ha good riddance blacklivesmatt,1
11852,no wer,1
1934,name woman fuck hard teen,1


In [ ]:
corpus = []
print(ds.shape[0])
for i in range(ds.shape[0]):
    corpus.append(ds.iloc[i][0])

7210


In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df2 = pd.DataFrame(denselist, columns=feature_names)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df2

,02,03,033m,05,06,0612,0620,06a,07000,07044,0750,07788427,088b,08a,0dya1i,0shares,0thbihdaycelebr,10,100,1000,1000a,1000x,1000yr,1001000,10014,10021,10025,100616,100a,100daysofcod,100happysongs,100juicedddddd,100k,100yr,101,1014,101dalmatian,1044,105kg,106,...,yyc,zaba,zach,zant,zanzibar,zara,zealand,zeegoldaward,zeeland,zeenew,zelda,zeldabreath,zen,zenblog,zero,zeroattack,zeuskodawn,zika,zilch,zille,zimbabw,zine,zionazi,zionism,zionist,zip,ziplin,zit,zomato,zomatoreview,zombi,zone,zoo,zootopia,zoro,zouma,zuma,zurich,zuricha,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tdf = df2
tdf['labely'] = list(ds.label)

tdf_hate = tdf[tdf.labely == 1]
tdf_nonhate = tdf[tdf.labely == 0]
X_train_hate = tdf_hate.sample(frac=0.9, random_state=0)
X_test_hate = tdf_hate.drop(X_train_hate.index)
X_train_nonhate =  tdf_nonhate.sample(frac=0.406, random_state=0)
X_test_nonhate = tdf_nonhate.drop(X_train_nonhate.index)
X_train_df = pd.concat([X_train_hate, X_train_nonhate], axis = 0)
X_train = X_train_df.drop(['labely'], axis = 1)
y_train = list(X_train_df.labely)
X_test_df = pd.concat([X_test_hate, X_test_nonhate], axis = 0)
X_test = X_test_df.drop(['labely'], axis = 1)
y_test = list(X_test_df.labely)

In [ ]:
# Logistic Regression
log_reg = LogisticRegression()
log_scores = cross_val_score(log_reg, X_train, y_train, cv=3)
log_reg_mean = log_scores.mean()

nav_clf = GaussianNB()
nav_scores = cross_val_score(nav_clf, X_train, y_train, cv=3)
nav_mean = nav_scores.mean()

rand_clf = RandomForestClassifier()
rand_score = cross_val_score(rand_clf, X_train, y_train, cv=3)
rand_mean = rand_score.mean()

dt_clf = tree.DecisionTreeClassifier()
dt_score = cross_val_score(dt_clf, X_train, y_train, cv=3)
dt_mean = dt_score.mean()

grad_clf = GradientBoostingClassifier()
grad_score = cross_val_score(grad_clf, X_train, y_train, cv=3)
grad_mean = grad_score.mean()



In [ ]:
d = {'Classifiers': ['Logistic Reg.', 'Naives Bayes','random forest','decision tree','gradient boosting'], 
    'Crossval Mean Scores': [log_reg_mean, nav_mean,rand_mean,dt_mean,grad_mean]}

result_df = pd.DataFrame(data=d)

In [ ]:
print(result_df)

         Classifiers  Crossval Mean Scores
0      Logistic Reg.              0.856010
1       Naives Bayes              0.745477
2      random forest              0.826518
3      decision tree              0.783643
4  gradient boosting              0.772491


In [ ]:
nav_clf.fit(X_train, y_train)
predict_nav = nav_clf.predict(X_test)

log_reg.fit(X_train, y_train)
predict_log = log_reg.predict(X_test)

rand_clf.fit(X_train, y_train)
predict = rand_clf.predict(X_test)

dt_clf.fit(X_train, y_train)
predict_dt = dt_clf.predict(X_test)

grad_clf.fit(X_train, y_train)
predict_grad = grad_clf.predict(X_test)

In [ ]:
print("Accuracy score")
print('Logistic Regression Score: ', accuracy_score(y_test, predict_log))
print('Naive Bayes Score: ', accuracy_score(y_test, predict_nav))
print('Random Forest Score: ', accuracy_score(y_test, predict))
print('Decision Tree Score: ', accuracy_score(y_test, predict_dt))
print('Grad Boosting Score: ', accuracy_score(y_test, predict_grad))

print("roc_accuracy_score")
print('Logistic Regression Score: ', roc_auc_score(y_test, predict_log))
print('Naive Bayes Score: ', roc_auc_score(y_test, predict_nav))
print('Random Forest Score: ', roc_auc_score(y_test, predict))
print('Decision Tree Score: ', roc_auc_score(y_test, predict_dt))
print('Grad Boosting Score: ', roc_auc_score(y_test, predict_grad))

Accuracy score
Logistic Regression Score:  0.8992125984251969
Naive Bayes Score:  0.6381102362204725
Random Forest Score:  0.9146456692913386
Decision Tree Score:  0.84
Grad Boosting Score:  0.9108661417322834
roc_accuracy_score
Logistic Regression Score:  0.8570846141743719
Naive Bayes Score:  0.7537525112552645
Random Forest Score:  0.8200072917170935
Decision Tree Score:  0.8107920438592245
Grad Boosting Score:  0.7540301108583046
